# Hopfield Net

In [1]:
import numpy as np

In [2]:
# Implementation for asynchronous Hopfield neural network
# Note: memory capacity ≃ 0.14*nodes
class Hopfield_Neural_Network:
    def __init__(self,nodes,iterations=100,weights=None):
        self.nodes = nodes
        self.iterations = iterations
        try:
            if weights == None:
                self.weights = np.zeros((nodes,nodes))
        except ValueError:
            self.weights = weights
    
    def store(self,input):
        dW = np.outer(input.transpose(),input)
        np.fill_diagonal(dW,0)
        self.weights += dW
        
    def recall(self,input):
        update_sequence = np.random.choice(self.nodes, self.iterations)
        for node in update_sequence:
            input[node] = np.sign(np.inner(input,self.weights[:,node]))
        return input

In [3]:
hnn = Hopfield_Neural_Network(10)

# Data

### Matrices

In [4]:
import re

In [5]:
class matrix_expansion:
    
    def __init__(self,active_motors):
        self.active_motors = active_motors
        active_sensors = np.array(active_motors*2)
        active_sensors[len(active_motors):] += 14
        self.active_sensors = active_sensors
        self.shape = ()
        
    def load_from_file(self,filename):
        f = open(filename,"r")
        matrix = f.read()
        f.close()
        matrix = re.split(",NEW_ROW,",matrix)
        matrix.pop()
        matrix = np.array([np.array(re.split(",", row)).astype(np.float) for row in matrix])
        self.shape = matrix.shape
        return matrix
        
    def reduced_matrix(self,matrix):
        matrix = matrix[:,self.active_sensors][self.active_motors]
        return matrix
    
    def expanded_matrix(self,reduced_matrix):
        matrix = np.zeros(self.shape)
        flat = reduced_matrix.flatten()
        matrix = np.zeros((14,28))
        k = 0
        for i in active_motors:
            for j in active_sensors:
                matrix[i,j] = flat[k]
                k += 1
        return matrix

In [6]:
active_motors = [1,3,4,5,10,12]
expander = matrix_expansion(active_motors)

In [7]:
# Front back
filename = "/home/markus/dep/dep_matrices/front_back.dep"
fb_matrix = expander.load_from_file(filename)
fb_reduced = expander.reduced_matrix(fb_matrix)
#fb_expanded = expander.expanded_matrix(fb_reduced)

In [8]:
# Front side
filename = "/home/markus/dep/dep_matrices/front_side.dep"
fs_matrix = expander.load_from_file(filename)
fs_reduced = expander.reduced_matrix(fs_matrix)
#fs_expanded = expander.expanded_matrix(fs_reduced)

In [9]:
# Side down
filename = "/home/markus/dep/dep_matrices/side_down.dep"
sd_matrix = expander.load_from_file(filename)
sd_reduced = expander.reduced_matrix(sd_matrix)
#sd_expanded = expander.expanded_matrix(sd_reduced)

In [10]:
matrices = {"fb": fb_reduced, "fs": fs_reduced, "sd": sd_reduced}

### Transition points

In [11]:
behaviors = ["fb","fs","sd"]
transitions = {("fb","fs"): [], ("fb","sd"): [], ("fs","fb"): [], ("fs","sd"): [], ("sd","fb"): [], ("sd","fb"): []}
transition_muscle_2 = [[-1.5,-1.5,-1.5,-1.5,-1.5,-1.5],[1,1,1,1,1,1]] # [pos,...],[direction,..] -- direction = 1 -> up, direction = 0 -> down

In [12]:
# need to get positions and velocities for fb, fs and sd at -1.5 going up

In [13]:
import pickle

In [14]:
fb = pickle.load(open("/home/markus/dep/dep_data/bases/fb.pickle","rb"))
fs = pickle.load(open("/home/markus/dep/dep_data/bases/fs.pickle","rb"))
sd = pickle.load(open("/home/markus/dep/dep_data/bases/sd.pickle", "rb"))

In [15]:
# obtained from plot, time indices that meat transition_muscle_2 condition
fb_t = 124
fs_t = 126
sd_t = 117

In [16]:
# pos data
pos = {"fb": fb[0][fb_t][active_motors], "fs": fs[0][fs_t][active_motors], "sd": sd[0][sd_t][active_motors]}
# vel data
vel = {"fb": fb[1][fb_t][active_motors], "fs": fs[1][fs_t][active_motors], "sd": sd[1][sd_t][active_motors]}

### Brain ID

In [17]:
# "fb" = [0.0,0.33], "fs" = [0.33, 0.66], "sd" = [0.66,1.00]
brain_id = {"fb": 1.0/6.0, "fs": 0.5, "sd": 5.0/6.0}

# Encoding

In [18]:
# HTM SDR Scalar Encoder
# Input: Scalar
# Parameters: n - number of units, w - bits used to represent signal (width), b - buckets (i.e. resolution), 
#             min - minimum value of input (inclusive), max - maximum input value (inclusive)
class scalar_sdr:
    
    def __init__(self, b, w, min_, max_, shape=0):
        if type(b) != int or type(w) != int or type(min_) != float or type(max_) != float:
            raise TypeError("b - buckets must be int, w - width must be int, min_ must be float and max_ must be float")
        self.b = b # must be int
        self.w = w # must be int
        self.min = min_ # must be float
        self.max = max_ # must be float
        self.n = b+w-1 # number of units for encoding
        self.ndarray_shape = shape
        
    def encode(self,input_):
        if input_ > self.max or input_ < self.min:
            raise ValueError("Input outside encoder range!")
        if type(input_) != float:
            raise TypeError("Input must be float!")
        output = np.zeros(self.n)-1
        index = int((input_-self.min)/(self.max-self.min)*self.b)
        output[index:index+self.w] = 1
        return output
    
    def encode_ndarray(self,input_):
        if input_.shape != self.ndarray_shape:
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        output = []
        for i in np.nditer(input_, order='K'):
            output.append(self.encode(float(i)))
        return np.array(output)

    def decode(self,input_):
        if len(input_) != self.n or len(np.nonzero(input_+1)[0]) != self.w:
            raise TypeError("Input does not correspond to encoder encoded data!")
        output = np.nonzero(input_+1)[0][0]/float(self.b)*(self.max-self.min)+self.min
        return output
    
    def decode_ndarray(self,input_):
        if input_.shape != (reduce(lambda x, y: x*y, self.ndarray_shape),self.n): 
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        input_ = input_.reshape(self.ndarray_shape+(self.n,))
        output = []
        for i in np.ndindex(self.ndarray_shape):
            output.append(self.decode(input_[i]))
        output = np.array(output).reshape(self.ndarray_shape)
        return output
    
    def set_ndarray_shape(self,shape):
        if type(shape) != tuple:
            raise TypeError("Must provide tuple of array dimensions!")
        self.ndarray_shape = shape

In [19]:
matrix_encoder = scalar_sdr(60,21,-0.3,0.3,(6,12))
pos_encoder = scalar_sdr(500,21,-100000.0,100000.0,(6,))
vel_encoder = scalar_sdr(200,41,-70.0,70.0,(6,))
brain_encoder = scalar_sdr(1000,101,0.0,1.0)

# Encoded data

In [20]:
encoded_data = {}
for transition in transitions:
    encoded_data[transition] = np.array([])
    m0 = matrix_encoder.encode_ndarray(matrices[transition[0]])
    encoded_data[transition] = np.append(encoded_data[transition], m0.flatten())
    m1 = matrix_encoder.encode_ndarray(matrices[transition[1]])
    encoded_data[transition] = np.append(encoded_data[transition], m1.flatten())
    p = pos_encoder.encode_ndarray(pos[transition[0]])
    encoded_data[transition] = np.append(encoded_data[transition], p.flatten())
    v = vel_encoder.encode_ndarray(vel[transition[0]])
    encoded_data[transition] = np.append(encoded_data[transition], v.flatten())
    id_ = brain_encoder.encode(brain_id[transition[1]])
    encoded_data[transition] = np.append(encoded_data[transition], id_.flatten())

# Train Hopfield